#word2vec 을 이용한 모델

- word2vec은 단어로 표현된 리스트를 입력값으로 넣어야 함
- 전처리된 텍스트를 불러온 후 각 단어들의 리스트로 나누어야 함

In [1]:
import os
import re
import pandas as pd
import numpy as np

In [2]:
DATA_IN_PATH = './data_in/'
DATA_OUT_PATH = './data_out/'
TRAIN_CLEAN_DATA = 'train_clean.csv'

train_data = pd.read_csv(DATA_IN_PATH + TRAIN_CLEAN_DATA)
train_data.head()

,review,sentiment
0,stuff going moment mj started listening music ...,1
1,classic war worlds timothy hines entertaining ...,1
2,film starts manager nicholas bell giving welco...,0
3,must assumed praised film greatest filmed oper...,0
4,superbly trashy wondrously unpretentious explo...,1


In [3]:
# 한 문장문장을 단어들로 변환
reviews = list(train_data['review'])
sentiments = list(train_data['sentiment'])

In [4]:
print(len(reviews))

25000


In [5]:
print(reviews[0])

stuff going moment mj started listening music watching odd documentary watched wiz watched moonwalker maybe want get certain insight guy thought really cool eighties maybe make mind whether guilty innocent moonwalker part biography part feature film remember going see cinema originally released subtle messages mj feeling towards press also obvious message drugs bad kay visually impressive course michael jackson unless remotely like mj anyway going hate find boring may call mj egotist consenting making movie mj fans would say made fans true really nice actual feature film bit finally starts minutes excluding smooth criminal sequence joe pesci convincing psychopathic powerful drug lord wants mj dead bad beyond mj overheard plans nah joe pesci character ranted wanted people know supplying drugs etc dunno maybe hates mj music lots cool things like mj turning car robot whole speed demon sequence also director must patience saint came filming kiddy bad sequence usually directors hate working

In [6]:
sentences = []
for review in reviews:
    sentences.append(review.split())

In [7]:
print(sentences[0])

['stuff', 'going', 'moment', 'mj', 'started', 'listening', 'music', 'watching', 'odd', 'documentary', 'watched', 'wiz', 'watched', 'moonwalker', 'maybe', 'want', 'get', 'certain', 'insight', 'guy', 'thought', 'really', 'cool', 'eighties', 'maybe', 'make', 'mind', 'whether', 'guilty', 'innocent', 'moonwalker', 'part', 'biography', 'part', 'feature', 'film', 'remember', 'going', 'see', 'cinema', 'originally', 'released', 'subtle', 'messages', 'mj', 'feeling', 'towards', 'press', 'also', 'obvious', 'message', 'drugs', 'bad', 'kay', 'visually', 'impressive', 'course', 'michael', 'jackson', 'unless', 'remotely', 'like', 'mj', 'anyway', 'going', 'hate', 'find', 'boring', 'may', 'call', 'mj', 'egotist', 'consenting', 'making', 'movie', 'mj', 'fans', 'would', 'say', 'made', 'fans', 'true', 'really', 'nice', 'actual', 'feature', 'film', 'bit', 'finally', 'starts', 'minutes', 'excluding', 'smooth', 'criminal', 'sequence', 'joe', 'pesci', 'convincing', 'psychopathic', 'powerful', 'drug', 'lord', 

conda install -c anaconda gensim

- num_features : 각 단어에 대해 임베딩된 벡터의 차원 지정(feature 수)
- min_word_count : 모델에 의미 있는 단어를 가지고 학습하기 위해 적은 빈도 수의 단어들은 학습하지 않기 위해 설정  
- num_workers : 모델 학습 시 학습을 위한 쓰레드 수 지정(기본값 3)  
- context : word2vec 을 수행하기 위한 컨텍스트 윈도우 사이즈 지정  
a. Maximum distance between the current and predicted word within a sentence.  
b. 기준 단어의 앞뒤에 존재하는 단어들로 기준 단어를 예측하게 되는데(sg=0, CBOW-Continuous Bag of Words)  
c. 이 때 기준 단어에서 앞뒤 얼마나 떨어져 있는 단어까지 고려하는가를 결정
- downsampling : word2vec 학습을 수행할 때 빠른 학습을 위해 정답 단어 레이블에 대한 다운샘플링 비율을 지정  
a. 보통 0.001이 좋은 성능을 낸다고 알려짐  
b. 0.001 값을 threshold 값으로 보고, 이 값보다 빈도수가 높은 단어들은 무작위로(랜덤) 다운샘플링 됨  
c. 빈도수가 높은 단어는 다운샘플링하여 가끔 학습(랜덤하게 무시)하고 빈도수가 낮은 단어는 출현 족족 학습하는 효과

In [8]:
num_features = 300
min_word_count = 40
num_workers = 4
context = 10
downsampling = 1e-3

In [9]:
# 어느 정도 수준까지 logging 을 할 것인지 (실제 현장에선 많이 쓴다 )

import logging
logging.basicConfig(format='%(asctime)s : $(levelname)s : %(message)s',
                   level=logging.INFO)

In [10]:
import smart_open
from gensim.models import word2vec

model = word2vec.Word2Vec(sentences, workers = num_workers, 
                         size = num_features, min_count = min_word_count,
                         window = context, sample = downsampling)

2021-10-08 14:16:56,403 : $(levelname)s : collecting all words and their counts
2021-10-08 14:16:56,404 : $(levelname)s : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2021-10-08 14:16:56,608 : $(levelname)s : PROGRESS: at sentence #10000, processed 1205223 words, keeping 51374 word types
2021-10-08 14:16:56,804 : $(levelname)s : PROGRESS: at sentence #20000, processed 2396605 words, keeping 67660 word types
2021-10-08 14:16:56,900 : $(levelname)s : collected 74065 word types from a corpus of 2988089 raw words and 25000 sentences
2021-10-08 14:16:56,901 : $(levelname)s : Loading a fresh vocabulary
2021-10-08 14:16:56,936 : $(levelname)s : effective_min_count=40 retains 8160 unique words (11% of original 74065, drops 65905)
2021-10-08 14:16:56,936 : $(levelname)s : effective_min_count=40 leaves 2627273 word corpus (87% of original 2988089, drops 360816)
2021-10-08 14:16:56,954 : $(levelname)s : deleting the raw counts dictionary of 74065 items
2021-10-08 14:16:56,956

In [11]:
# 여기까지 한 내용 저장하기
model_name = "300features_40minwords_10context"
model.save(model_name)

2021-10-08 14:17:06,809 : $(levelname)s : saving Word2Vec object under 300features_40minwords_10context, separately None
2021-10-08 14:17:06,810 : $(levelname)s : not storing attribute vectors_norm
2021-10-08 14:17:06,811 : $(levelname)s : not storing attribute cum_table
2021-10-08 14:17:06,967 : $(levelname)s : saved 300features_40minwords_10context


In [12]:
# 선형 회귀 모델을 훈련시키기

- 위에서 많은 word2vec 모델을 활용하여 선형 회귀 모델을 학습시켜봄
- 각 리뷰를 같은 형태의 입력값으로 만들어야 함
- 기뷰마다 단어의 수가 모두 다르므로 입력값을 하나의 형태로 만듬
- 가장 단순한 방법으로, 문장에 있는 모든 단어의 벡터값에 대한 평균을 내서 리뷰 하나당 하나의 벡터로 만드는 방법을 사용하겠음

In [13]:
def get_features(words, model, num_features):
    feature_vector = np.zeros((num_features), dtype = np.float32)
    
    num_words = 0
    index2word_set = set(model.wv.index2word) # index2word : 단어 사전
    
    for w in words:  # 단어를 하나씩 꺼내서 단어 사전에 존재한다면, 추가시키고, 벡터값을 꺼내서 곱해준다 
        if w in index2word_set:
            num_words += 1
            feature_vector = np.add(feature_vector, model[w])
            
        feature_vector = np.divide(feature_vector, num_words) # 단어별로 벡터값이 있는데, 이걸 다 더해서 평균을 낸 것
        return feature_vector

- words : 단어의 모음인 하나의 리뷰가 들어감
- model : word2vec 모델
- num_features : word2vec 으로 임베딩할 때 정했던 벡터의 차원 수  
- 결극 하나의 문장에 등장하는 사전에 등록된 단어들의 벡터값의 평균을 구함

In [14]:
def get_dataset(review, model, num_features):
    dataset = list()
    
    for s in reviews:
        dataset.append(get_features(words, model, num_features))
        # 이렇게 해서 나온 평균 벡터값을 계속 추가해 준다. 
        
        reveiwFeatureVecs = np.stack(dataset)
        
        return reveiwFeatureVecs

- review : 전체 리뷰 데이터  
- model : 학습시킨 모델  
- num_features : word2vec 임베딩 시 정했던 벡터의 차원 수  
- np.stack(dataset, axis=0) 은 row 로 데이터를 쌓으면서 numpy 배열을 만든다는 의미  
- 이렇게 하여 row가 전체 샘플 수 만큼, column 은 feature의 차원 수가 됨  

In [16]:
train_data_vecs = get_dataset(sentences, model, num_features)

NameError: name 'words' is not defined

In [ ]:
from sklearn.model_selection import train_test_
from sklearn.linear_model import LogisticRegression

lgs = LogisticRegression(class_weight='balanced')
lgs.fit(X_train, y_train)

In [ ]:
predicted = lgs.predict(X_test)
print('Accuracy : %f' % lgs.socre(X_test, y_test))

In [ ]:
TEST_CLEAN_DATA = 'test_clean.csv'

test_data = pd.read_csv(DATA_IN_PATH + TEST_CLEAN_DATA)
test_review = list(test_data['review'])

In [ ]:
test_sentences = list()
for review in test_review:
    test_sentences.append(review.split())

In [ ]:
test_data_vecs = get_dataset(test_sentences, model, num_feature)

In [ ]:
test_predicted = lgs.predict(test_data_vecs)

ids = list(test_data['id'])
answer_dataset = pd.DataFrame({'id': ids, 'sentiment': test_predicted})

In [ ]:
if not os.path.exists(DATA_OUT_PATH):
    